In [1]:
import json
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageOps
import matplotlib.patches as patches
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split
import random

In [2]:
def convert_coco_to_yolo(image_ids, images, annotations, categories, category_name_to_id, images_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)

    for image_id in image_ids:
        image_info = images[image_id]
        image_filename = image_info['file_name']
        image_width = image_info['width']
        image_height = image_info['height']
        image_path = os.path.join(images_dir, image_filename)

        if not os.path.exists(image_path):
            continue

        output_image_path = os.path.join(output_dir, 'images', image_filename)
        output_label_path = os.path.join(output_dir, 'labels', os.path.splitext(image_filename)[0] + '.txt')

        shutil.copyfile(image_path, output_image_path)

        image_annotations = [ann for ann in annotations if ann['image_id'] == image_id]
        with open(output_label_path, 'a') as f:
            for annotation in image_annotations:
                category_name = categories[annotation['category_id']]['name']
                category_id = category_name_to_id[category_name]
                
                # Normalize bounding box coordinates
                bbox = annotation['bbox']
                bbox[0] /= image_width  # x
                bbox[1] /= image_height # y
                bbox[2] /= image_width  # width
                bbox[3] /= image_height # height
                bbox_str = ' '.join(map(str, bbox))

                # Normalize segmentation points
                segmentation = annotation['segmentation'][0]
                segmentation = np.array(segmentation).reshape((-1, 2))
                segmentation[:, 0] /= image_width   # x points
                segmentation[:, 1] /= image_height  # y points
                segmentation_str = ' '.join(map(str, segmentation.flatten()))

                f.write(f"{category_id} {bbox_str} {segmentation_str}\n")

In [3]:
# Paths to your data
coco_annotations_path = 'SAF22_COCO_CorentinDoche.json'
images_dir = 'images'  # Change this to your image directory
output_dir = 'output'  # Directory where YOLO formatted data will be saved

# Load COCO annotations
with open(coco_annotations_path, 'r') as f:
    data = json.load(f)

images = {image['id']: image for image in data['images']}
annotations = data['annotations']
categories = {category['id']: category for category in data['categories']}
category_name_to_id = {category['name']: idx for idx, category in enumerate(categories.values())}

# Collect all image IDs
all_image_ids = list(images.keys())

# Split data into train and validation sets
train_image_ids, val_image_ids = train_test_split(all_image_ids, test_size=0.2, random_state=42)

# Define paths for train and validation
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')

# Convert and save the training data
convert_coco_to_yolo(train_image_ids, images, annotations, categories, category_name_to_id, images_dir, train_dir)

# Convert and save the validation data
convert_coco_to_yolo(val_image_ids, images, annotations, categories, category_name_to_id, images_dir, val_dir)

print("Data converted and split into train and val sets successfully!")


Data converted and split into train and val sets successfully!


In [3]:
from ultralytics import YOLO

In [5]:
# load a pretrained model (recommended for training)
model = YOLO("yolov8n-seg.pt")  

In [6]:
# Train the model
results = model.train(data="C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml", epochs=5, imgsz=640)

Ultralytics YOLOv8.2.61  Python-3.12.4 torch-2.3.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train21, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ds678 (ds678-university-of-sussex). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\labels... 4

train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_13.png: 2 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_14.png: 2 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_22.png: 102 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_23.png: 94 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_25.png: 48 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_26.png: 32 duplicate labels removed

...

                   all         11        438     0.0718      0.499      0.165     0.0676          0          0          0          0



5 epochs completed in 0.059 hours.
Optimizer stripped from runs\segment\train21\weights\last.pt, 6.8MB
Optimizer stripped from runs\segment\train21\weights\best.pt, 6.8MB

Validating runs\segment\train21\weights\best.pt...
Ultralytics YOLOv8.2.61  Python-3.12.4 torch-2.3.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
YOLOv8n-seg summary (fused): 195 layers, 3,258,844 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all         11        438     0.0715      0.495      0.164      0.067          0          0          0          0
  ClassicalEvaporation         11        226     0.0934      0.412      0.194     0.0786          0          0          0          0
    TransitionalMixing         11        124     0.0743      0.597      0.164     0.0732          0          0          0          0
       DiffusiveMixing          7         88     0.0469      0.477      0.135     0.0492          0          0          0          0
Speed: 4.3ms preprocess, 123.9ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs\segment\train21


lr/pg0,▁▆█▇▃
lr/pg1,▁▆█▇▃
lr/pg2,▁▆█▇▃
metrics/mAP50(B),▁▂▃▅█
metrics/mAP50(M),▁▁▁▁▁
metrics/mAP50-95(B),▁▂▃▅█
metrics/mAP50-95(M),▁▁▁▁▁
metrics/precision(B),▁▅▃▂█
metrics/precision(M),▁▁▁▁▁
metrics/recall(B),▁▇▆▅█
metrics/recall(M),▁▁▁▁▁


In [7]:
# print(results.class_result(0))
# print(results.class_result(1))
# print(results.class_result(2))
# print(results.class_result(3))

In [8]:
# Assuming 'results' is an instance of a class that has the 'class_result' method
class_names = ['ClassicalEvaporation', 'TransitionalMixing', 'DiffusiveMixing', 'Spray']

num_classes = len(class_names)

for i in range(num_classes):
    try:
        result = results.class_result(i)
        print(f"Class {i} ({class_names[i]}): {result}")
    except IndexError as e:
        print(f"IndexError: {e} for class index {i} ({class_names[i]})")


Class 0 (ClassicalEvaporation): (0.09337349397590361, 0.41150442477876104, 0.19374558982464943, 0.07858559536425513, 0.0, 0.0, 0.0, 0.0)
Class 1 (TransitionalMixing): (0.07429718875502007, 0.5967741935483871, 0.16403307904492542, 0.07320384999127037, 0.0, 0.0, 0.0, 0.0)
Class 2 (DiffusiveMixing): (0.046875, 0.4772727272727273, 0.13503058467352202, 0.04916365426457621, 0.0, 0.0, 0.0, 0.0)
IndexError: index 3 is out of bounds for axis 0 with size 3 for class index 3 (Spray)


In [9]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

acc,▁▆▆█████
loss,█▇█▅▂▂▃▁
acc,0.88894
loss,0.07799


In [12]:
import json
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image, ImageEnhance, ImageOps
import random
import matplotlib.pyplot as plt
import wandb
from ultralytics import YOLO

def augment_image(image):
    # Change color
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(random.uniform(0.8, 1.2))

    # Rotation
    angle = random.choice([0, 90, 180, 270])
    image = image.rotate(angle)

    # Flipping
    if random.choice([True, False]):
        image = ImageOps.mirror(image)

    return image

def convert_coco_to_yolo(image_ids, images, annotations, categories, category_name_to_id, images_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)

    for image_id in image_ids:
        image_info = images[image_id]
        image_filename = image_info['file_name']
        image_width = image_info['width']
        image_height = image_info['height']
        image_path = os.path.join(images_dir, image_filename)

        if not os.path.exists(image_path):
            continue

        image = Image.open(image_path)
        image = augment_image(image)

        output_image_path = os.path.join(output_dir, 'images', image_filename)
        output_label_path = os.path.join(output_dir, 'labels', os.path.splitext(image_filename)[0] + '.txt')

        image.save(output_image_path)

        image_annotations = [ann for ann in annotations if ann['image_id'] == image_id]
        with open(output_label_path, 'a') as f:
            for annotation in image_annotations:
                category_name = categories[annotation['category_id']]['name']
                category_id = category_name_to_id[category_name]
                
                # Normalize bounding box coordinates
                bbox = annotation['bbox']
                bbox[0] /= image_width  # x
                bbox[1] /= image_height # y
                bbox[2] /= image_width  # width
                bbox[3] /= image_height # height
                bbox_str = ' '.join(map(str, bbox))

                # Normalize segmentation points
                segmentation = annotation['segmentation'][0]
                segmentation = np.array(segmentation).reshape((-1, 2))
                segmentation[:, 0] /= image_width   # x points
                segmentation[:, 1] /= image_height  # y points
                segmentation_str = ' '.join(map(str, segmentation.flatten()))

                f.write(f"{category_id} {bbox_str} {segmentation_str}\n")

# Paths to your data
coco_annotations_path = 'SAF22_COCO_CorentinDoche.json'
images_dir = 'images'  # Change this to your image directory
output_dir = 'output'  # Directory where YOLO formatted data will be saved

# Load COCO annotations
with open(coco_annotations_path, 'r') as f:
    data = json.load(f)

images = {image['id']: image for image in data['images']}
annotations = data['annotations']
categories = {category['id']: category for category in data['categories']}
category_name_to_id = {category['name']: idx for idx, category in enumerate(categories.values())}

# Collect all image IDs
all_image_ids = list(images.keys())

# Split data into train and validation sets
train_image_ids, val_image_ids = train_test_split(all_image_ids, test_size=0.2, random_state=42)

# Define paths for train and validation
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')

# Convert and save the training data
convert_coco_to_yolo(train_image_ids, images, annotations, categories, category_name_to_id, images_dir, train_dir)

# Convert and save the validation data
convert_coco_to_yolo(val_image_ids, images, annotations, categories, category_name_to_id, images_dir, val_dir)

print("Data converted and split into train and val sets successfully!")

# Load and train the model before tuning
model = YOLO("yolov8n-seg.pt")
results_before = model.train(data="C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml", epochs=5, imgsz=640)

# Hyperparameter tuning and training
model_tuned = YOLO("yolov8n-seg.pt")

# Example of hyperparameters tuning (change as needed)
tuned_results = model_tuned.train(data="C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml", epochs=10, imgsz=640, lr0=0.01, momentum=0.9)

# Save the tuned model
tuned_model_dir = 'tuned_model'
os.makedirs(tuned_model_dir, exist_ok=True)
tuned_model_path = os.path.join(tuned_model_dir, 'best.pt')
model_tuned.save(tuned_model_path)

# Compare results before and after tuning
class_names = ['ClassicalEvaporation', 'TransitionalMixing', 'DiffusiveMixing', 'Spray']
num_classes = len(class_names)

print("\nResults before tuning:")
for i in range(num_classes):
    try:
        result = results_before.class_result(i)
        print(f"Class {i} ({class_names[i]}): {result}")
    except IndexError as e:
        print(f"IndexError: {e} for class index {i} ({class_names[i]})")

print("\nResults after tuning:")
for i in range(num_classes):
    try:
        result = tuned_results.class_result(i)
        print(f"Class {i} ({class_names[i]}): {result}")
    except IndexError as e:
        print(f"IndexError: {e} for class index {i} ({class_names[i]})")

# Initialize wandb for logging
wandb.init(
    project="my-awesome-project",
    config={
        "learning_rate": 0.01,
        "architecture": "YOLOv8",
        "dataset": "SAF_Project",
        "epochs": 10,
    }
)

# Log metrics to wandb
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    wandb.log({"acc": acc, "loss": loss})

# Finish the wandb run
wandb.finish()


Data converted and split into train and val sets successfully!
Ultralytics YOLOv8.2.61  Python-3.12.4 torch-2.3.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train24, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=N

Freezing layer 'model.22.dfl.conv.weight'


train: Scanning C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\labels... 4

train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_13.png: 4 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_14.png: 4 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_22.png: 204 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_23.png: 188 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_25.png: 96 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_26.png: 64 duplicate labels remove


val: Scanning C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\val\labels... 11 im

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all         11        438     0.0897      0.517      0.191      0.103          0          0          0          0
  ClassicalEvaporation         11        226      0.112      0.398      0.195      0.108          0          0          0          0
    TransitionalMixing         11        124      0.101      0.629      0.283      0.146          0          0          0          0
       DiffusiveMixing          7         88     0.0562      0.523     0.0961     0.0544          0          0          0          0
Speed: 4.7ms preprocess, 116.7ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs\segment\train25


lr/pg0,▁▄▆▇███▆▅▂
lr/pg1,▁▄▆▇███▆▅▂
lr/pg2,▁▄▆▇███▆▅▂
metrics/mAP50(B),▁▂▄▇▇▇████
metrics/mAP50(M),▁▁▁▁▁▁▁▁▁▁
metrics/mAP50-95(B),▁▂▄▇▇▇████
metrics/mAP50-95(M),▁▁▁▁▁▁▁▁▁▁
metrics/precision(B),▁▂▃▄▆▆▆▇▇█
metrics/precision(M),▁▁▁▁▁▁▁▁▁▁
metrics/recall(B),▁▄▆▆█▇▆▆▆▆
metrics/recall(M),▁▁▁▁▁▁▁▁▁▁



Results before tuning:
Class 0 (ClassicalEvaporation): (0.09315589353612168, 0.4336283185840708, 0.250085389285505, 0.12278906420411136, 0.0, 0.0, 0.0, 0.0)
Class 1 (TransitionalMixing): (0.0794148380355277, 0.6129032258064516, 0.1611141367157776, 0.06536550694223706, 0.0, 0.0, 0.0, 0.0)
Class 2 (DiffusiveMixing): (0.048859934853420196, 0.5113636363636364, 0.07180847931152634, 0.034011838849985994, 0.0, 0.0, 0.0, 0.0)
IndexError: index 3 is out of bounds for axis 0 with size 3 for class index 3 (Spray)

Results after tuning:
Class 0 (ClassicalEvaporation): (0.11221945137157108, 0.39823008849557523, 0.1953125194679629, 0.10764474205390859, 0.0, 0.0, 0.0, 0.0)
Class 1 (TransitionalMixing): (0.10051546391752578, 0.6290322580645161, 0.2829217896545582, 0.1463492875301516, 0.0, 0.0, 0.0, 0.0)
Class 2 (DiffusiveMixing): (0.05623471882640587, 0.5227272727272727, 0.0960832671620879, 0.05440473375836739, 0.0, 0.0, 0.0, 0.0)
IndexError: index 3 is out of bounds for axis 0 with size 3 for class 

acc,▁▇▇▇█▇██
loss,█▅▃▃▃▁▂▂
acc,0.81103
loss,0.20133


In [1]:
model.predict(path ="C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\val\images" ,show=True, save=True, hide_labels=False, hide_conf=False, conf=0.5, save_txt=False, save_crop=False, line_thickness=2)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1280568635.py, line 1)

In [5]:
from ultralytics import YOLO

# Load the model
model = YOLO("yolov8n-seg.pt")

# Perform predictions
model.predict(
    source=r"C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\val\images\ABTCWS002_frame_23.png",
    show=True,
    save=True,
    show_labels=True,
    show_conf=True,
    conf=0.5,
    save_txt=False,
    save_crop=False,
    line_width=2
)




image 1/1 C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\val\images\ABTCWS002_frame_23.png: 480x640 (no detections), 125.6ms
Speed: 0.0ms preprocess, 125.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\segment\predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [2]:
# load a pretrained model (recommended for training)
model = YOLO("yolov8m-seg.pt") 

100%|█████████████████████████████████████████████████████████████████████████████| 52.4M/52.4M [00:02<00:00, 24.4MB/s]


In [3]:
# Train the model
results = model.train(data="C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml", epochs=5, imgsz=640)

New https://pypi.org/project/ultralytics/8.2.63 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.61  Python-3.12.4 torch-2.3.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train26, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ds678 (ds678-university-of-sussex). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\labels... 4

train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_13.png: 4 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_14.png: 5 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_22.png: 255 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_23.png: 235 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_25.png: 120 duplicate labels removed
train: WARNING  C:\Users\devan\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\train\images\ABCCWS001_frame_26.png: 80 duplicate labels remov

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP

WARNING  NMS time limit 2.550s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all         11        438      0.208      0.305      0.195     0.0943          0          0          0          0
  ClassicalEvaporation         11        226      0.129      0.358      0.185     0.0928          0          0          0          0
    TransitionalMixing         11        124       0.45      0.476      0.357      0.173          0          0          0          0
       DiffusiveMixing          7         88     0.0451     0.0795     0.0416     0.0169          0          0          0          0
Speed: 1.8ms preprocess, 678.0ms inference, 0.0ms loss, 241.7ms postprocess per image
Results saved to runs\segment\train26


lr/pg0,▁▆█▇▃
lr/pg1,▁▆█▇▃
lr/pg2,▁▆█▇▃
metrics/mAP50(B),▁▇█▅█
metrics/mAP50(M),▁▁▁▁▁
metrics/mAP50-95(B),▁▆█▅█
metrics/mAP50-95(M),▁▁▁▁▁
metrics/precision(B),▁▆█▆█
metrics/precision(M),▁▁▁▁▁
metrics/recall(B),▆▁█▃█
metrics/recall(M),▁▁▁▁▁


In [4]:
# Initialize wandb for logging
wandb.init(
    project="my-awesome-project",
    config={
        "learning_rate": 0.01,
        "architecture": "YOLOv8",
        "dataset": "SAF_Project",
        "epochs": 10,
    }
)

# Log metrics to wandb
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    wandb.log({"acc": acc, "loss": loss})

# Finish the wandb run
wandb.finish()

acc,▁▆▅▇▆███
loss,█▄▃▃▂▁▁▂
acc,0.83
loss,0.17314
